In [2]:
import json
import csv
import os
import pandas as pd

class JSONToCSVConverter:
    def __init__(self, filename):
        """
        Initialize the converter with a specific filename
        
        :param filename: Path to the CSV file
        """
        self.filename = filename
        
        # Ensure the file exists with headers if it doesn't
        self.ensure_file_exists()
    
    def ensure_file_exists(self):
        """
        Ensure the CSV file exists with proper headers
        """
        # Create the file with headers if it doesn't exist
        if not os.path.exists(self.filename):
            with open(self.filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['question', 'answer'])
    
    def upload_and_append(self, existing_file_path, additional_json_data=None):
        """
        Upload an existing CSV or JSON file and optionally append additional JSON data
        
        :param existing_file_path: Path to the existing file to upload
        :param additional_json_data: Optional JSON string to append to the file
        :return: Total number of entries after upload and append
        """
        try:
            # Determine file type
            file_ext = os.path.splitext(existing_file_path)[1].lower()
            
            # Read the existing file
            if file_ext == '.csv':
                # Read existing CSV
                df = pd.read_csv(existing_file_path)
            elif file_ext == '.json':
                # Read existing JSON
                with open(existing_file_path, 'r', encoding='utf-8') as f:
                    df = pd.DataFrame(json.load(f))
            else:
                print(f"Unsupported file type: {file_ext}")
                return 0
            
            # Ensure the dataframe has 'question' and 'answer' columns
            if 'question' not in df.columns or 'answer' not in df.columns:
                print("Error: File must have 'question' and 'answer' columns")
                return 0
            
            # Select only required columns
            df = df[['question', 'answer']]
            
            # Append to existing file
            df.to_csv(self.filename, mode='a', header=False, index=False)
            
            # If additional JSON data is provided, add it
            if additional_json_data:
                # Parse the JSON input
                entries = json.loads(additional_json_data)
                
                # Validate the input is a list
                if not isinstance(entries, list):
                    print("Error: JSON input must be a list of entries")
                    return len(df)
                
                # Validate and append additional entries
                valid_entries = []
                for entry in entries:
                    if 'question' not in entry or 'answer' not in entry:
                        print(f"Error: Invalid entry skipped - {entry}")
                        continue
                    valid_entries.append([entry['question'], entry['answer']])
                
                # Append valid entries to the CSV file
                with open(self.filename, 'a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerows(valid_entries)
                
                print(f"Successfully added {len(valid_entries)} additional entries!")
            
            # Return total number of entries
            return len(df) + len(valid_entries) if 'valid_entries' in locals() else len(df)
        
        except Exception as e:
            print(f"An error occurred: {e}")
            return 0
    
    def view_entries(self, num_entries=None):
        """
        View entries in the CSV file
        
        :param num_entries: Number of entries to view (None for all)
        """
        try:
            with open(self.filename, 'r', newline='', encoding='utf-8') as csvfile:
                reader = list(csv.reader(csvfile))
                
                # Skip header
                entries = reader[1:]
                
                # Limit entries if specified
                if num_entries is not None:
                    entries = entries[:num_entries]
                
                for i, row in enumerate(entries, 1):
                    print(f"Entry {i}: Question: {row[0]}, Answer: {row[1]}")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    def get_entry_count(self):
        """
        Get the number of entries in the CSV file
        
        :return: Number of entries (excluding header)
        """
        try:
            with open(self.filename, 'r', newline='', encoding='utf-8') as csvfile:
                return len(list(csv.reader(csvfile))) - 1
        except Exception as e:
            print(f"An error occurred: {e}")
            return 0

def main():
    # Specify the full path to your target CSV file
    target_file_path = '/kaggle/working/faq_data.csv'
    
    # Specify the path to your existing file to upload
    existing_file_path = '/kaggle/input/dataset/ev_data.csv'  
    
    # Create the converter
    converter = JSONToCSVConverter(target_file_path)
    
    # Print current number of entries before upload
    print(f"Current entries before upload: {converter.get_entry_count()}")
    
    # Optional: Additional JSON data to append
    additional_json = json.dumps([
    {
    "question": "How many electric vehicles were sold globally in 2024?",
    "answer": "Approximately 17.1 million EVs were sold worldwide in 2024, representing a 25% increase from the previous year."
  },
  {
    "question": "What percentage of total car sales did EVs account for globally in 2024?",
    "answer": "Electric cars accounted for about 20% of total global car sales in 2024."
  },
  {
    "question": "How many electric vehicles were sold in China in 2024?",
    "answer": "China sold approximately 12.87 million passenger electric vehicles in 2024, with 60% being battery electric vehicles (BEVs) and 40% plug-in hybrid electric vehicles (PHEVs)."
  },
  {
    "question": "What was the market share of plug-in electric vehicles in China in 2024?",
    "answer": "Plug-in electric vehicles accounted for 47.9% of the overall automotive sales in China in 2024."
  },
  {
    "question": "How many electric vehicles were sold in the United States in 2024?",
    "answer": "About 1.7 million EVs were sold in the U.S. in 2024, a 21% increase from the 1.4 million sold in 2023."
  },
  {
    "question": "What percentage of new car sales in the U.S. were electric in 2024?",
    "answer": "Electric vehicles accounted for approximately 10.2% of all new car sales in the U.S. in 2024."
  },
  {
    "question": "What was Tesla's market share in the U.S. EV market in 2024?",
    "answer": "Tesla's U.S. EV market share fell to 44% in 2024."
  },
  {
    "question": "How much did lithium-ion battery prices decrease by 2024?",
    "answer": "Lithium-ion battery prices have fallen significantly since 2010, contributing to the decreasing cost of EVs. Specific percentage reductions vary by source."
  },
  {
    "question": "What is the average range of a new electric vehicle in 2024?",
    "answer": "The average range of a new EV in 2024 is approximately 250-300 miles per charge."
  },
  {
    "question": "Which electric vehicle has the longest range in 2024?",
    "answer": "The Lucid Air Grand Touring has an EPA-estimated range of 516 miles per charge."
  },
  {
    "question": "How quickly can some EVs charge in 2024?",
    "answer": "Some EVs can charge from 10% to 80% in under 20 minutes using ultra-fast DC chargers."
  },
  {
    "question": "How many public charging stations are there worldwide in 2024?",
    "answer": "There are over 3 million public charging points globally as of 2024."
  },
  {
    "question": "What is the energy efficiency of electric vehicles compared to gasoline cars?",
    "answer": "EVs convert about 85-90% of electrical energy into motion, compared to 20-30% for gasoline cars."
  },
  {
    "question": "What is the typical lifespan of an EV battery in 2024?",
    "answer": "Most EV batteries last between 100,000-200,000 miles, with some exceeding 300,000 miles."
  },
  {
    "question": "What incentives are available for EV purchases in the U.S. in 2024?",
    "answer": "The U.S. offers tax credits up to $7,500 for eligible EV purchases in 2024."
  },
  {
    "question": "How much can EV owners in the U.S. save on fuel annually compared to gasoline car owners?",
    "answer": "EV owners in the U.S. save an average of $800-1,000 per year on fuel compared to gasoline car owners."
  },
  {
    "question": "Which company became the largest EV manufacturer in 2024?",
    "answer": "BYD surpassed Tesla in Q4 2024 as the world's top EV seller, delivering over 526,000 EVs."
  },
  {
    "question": "What is the most affordable electric vehicle in 2024?",
    "answer": "The Chevrolet Bolt remains one of the most affordable EVs in 2024, with prices starting around $26,500."
  },
  {
    "question": "What percentage of an EV battery's materials can be recycled?",
    "answer": "Over 95% of an EV battery’s materials can be recycled."
  },
  {
    "question": "What was the market share of electric vehicles in Norway in 2024?",
    "answer": "In Norway, electric vehicles accounted for approximately 91.6% of new car sales in 2024."
  },
    {
    "question": "What is an electric vehicle (EV)?",
    "answer": "An electric vehicle (EV) is a car powered by electricity instead of gasoline or diesel. It uses a battery and an electric motor for propulsion."
  },
  {
    "question": "What are the main types of electric vehicles?",
    "answer": "The main types of EVs are Battery Electric Vehicles (BEVs), Plug-in Hybrid Electric Vehicles (PHEVs), and Hybrid Electric Vehicles (HEVs)."
  },
  {
    "question": "How long does it take to charge an EV?",
    "answer": "Charging time varies by charger type. Level 1 (home outlet) takes 8-20 hours, Level 2 (public/home charger) takes 4-8 hours, and DC fast charging takes 20-60 minutes."
  },
  {
    "question": "What is the average range of an EV?",
    "answer": "Most EVs today have a range of 250-300 miles per charge, with some models exceeding 400 miles."
  },
  {
    "question": "How much does it cost to charge an electric car?",
    "answer": "The cost varies by electricity rates but typically ranges from $5 to $15 for a full charge at home, and slightly higher at public fast chargers."
  },
  {
    "question": "Are electric cars really better for the environment?",
    "answer": "Yes, EVs produce zero tailpipe emissions and have lower lifetime carbon footprints than gas cars, especially when charged with renewable energy."
  },
  {
    "question": "How long do EV batteries last?",
    "answer": "Most EV batteries last between 100,000 to 200,000 miles, with some exceeding 300,000 miles before noticeable degradation."
  },
  {
    "question": "Can I charge an EV at home?",
    "answer": "Yes, most EV owners charge at home using a standard outlet (Level 1) or a faster Level 2 home charger."
  },
  {
    "question": "Are there enough charging stations for EVs?",
    "answer": "Public charging infrastructure is expanding rapidly, with over 3 million public chargers worldwide as of 2024."
  },
  {
    "question": "Do electric cars require less maintenance?",
    "answer": "Yes, EVs have fewer moving parts, no oil changes, and less wear on brakes, making them cheaper to maintain than gas cars."
  },
  {
    "question": "What happens if an EV runs out of charge?",
    "answer": "If an EV runs out of charge, it must be towed to a charging station, similar to how a gas car needs refueling when empty."
  },
  {
    "question": "Are electric cars more expensive than gasoline cars?",
    "answer": "Upfront costs can be higher, but tax incentives, lower fuel costs, and reduced maintenance make EVs more cost-effective in the long run."
  },
  {
    "question": "How fast can an electric vehicle go?",
    "answer": "EVs can be very fast, with some high-performance models like the Tesla Model S Plaid reaching 0-60 mph in under 2 seconds."
  },
  {
    "question": "Can I take an EV on a road trip?",
    "answer": "Yes, many EVs have long ranges, and fast-charging networks like Tesla Superchargers and Electrify America make road trips feasible."
  },
  {
    "question": "What incentives are available for buying an EV?",
    "answer": "In the U.S., federal tax credits up to $7,500 are available, along with state and local incentives depending on your location."
  },
  {
    "question": "Are electric cars safer than gas-powered cars?",
    "answer": "Yes, EVs often have lower centers of gravity, fewer flammable components, and strong safety ratings due to crash-resistant designs."
  },
  {
    "question": "Do electric cars work in cold weather?",
    "answer": "Yes, but cold weather can reduce battery efficiency and range, though newer EVs have battery management systems to mitigate this."
  },
  {
    "question": "Can you charge an EV in the rain?",
    "answer": "Yes, EV chargers are designed to be safe for use in all weather conditions, including rain."
  },
  {
    "question": "How much does it cost to replace an EV battery?",
    "answer": "Battery replacement costs range from $5,000 to $15,000, but most manufacturers offer long warranties (8-10 years) on batteries."
  },
  {
    "question": "What is the future of electric vehicles?",
    "answer": "EV adoption is growing rapidly, with advancements in battery technology, charging networks, and government policies accelerating the transition to electric mobility."
  },
    {
    "question": "Can I install an EV charger at home?",
    "answer": "Yes, you can install a Level 2 home charger for faster charging. Installation costs range from $500 to $2,000, depending on your electrical setup."
  },
  {
    "question": "Do EVs have gears?",
    "answer": "Most EVs have a single-speed transmission because electric motors provide instant torque and do not require multiple gears like gas engines."
  },
  {
    "question": "How much does it cost to own an EV compared to a gas car?",
    "answer": "While EVs may have a higher upfront cost, they save money on fuel, maintenance, and tax incentives, making them more affordable over time."
  },
  {
    "question": "Can EV batteries be recycled?",
    "answer": "Yes, over 95% of an EV battery’s materials, including lithium, cobalt, and nickel, can be recycled or repurposed."
  },
  {
    "question": "Do EVs lose charge when parked for long periods?",
    "answer": "Yes, but very slowly. Most modern EVs lose only a small percentage of charge per day when not in use."
  },
  {
    "question": "Can I charge my EV with solar panels?",
    "answer": "Yes, many EV owners install solar panels to generate clean energy for charging their vehicles, reducing electricity costs."
  },
  {
    "question": "What is regenerative braking in an EV?",
    "answer": "Regenerative braking allows EVs to recover energy when slowing down, converting it back into battery power to improve efficiency."
  },
  {
    "question": "How do electric cars perform compared to gasoline cars?",
    "answer": "EVs offer instant acceleration, smoother rides, and lower center of gravity for better handling compared to gasoline cars."
  },
  {
    "question": "Are all EVs fully electric?",
    "answer": "No, some EVs are Plug-in Hybrid Electric Vehicles (PHEVs) that use both electricity and gasoline for extended range."
  },
  {
    "question": "Can an EV battery catch fire?",
    "answer": "While rare, EV batteries can catch fire due to thermal runaway, but incidents are less common than gas car fires."
  },
  {
    "question": "Can I tow with an electric vehicle?",
    "answer": "Yes, many EVs like the Rivian R1T and Ford F-150 Lightning offer strong towing capabilities, but towing reduces range."
  },
  {
    "question": "What happens to old EV batteries?",
    "answer": "Old EV batteries can be repurposed for energy storage or recycled to extract valuable materials for new batteries."
  },
  {
    "question": "How many miles per gallon equivalent (MPGe) do EVs get?",
    "answer": "EVs have an MPGe rating, typically between 90-140 MPGe, which represents their efficiency compared to gasoline cars."
  },
  {
    "question": "Can I drive an EV in the snow?",
    "answer": "Yes, EVs handle well in snow, especially all-wheel-drive models. However, cold weather may reduce battery efficiency."
  },
  {
    "question": "Do electric cars make noise?",
    "answer": "EVs are much quieter than gas cars, but many have artificial sounds at low speeds to alert pedestrians."
  },
  {
    "question": "Can I use a regular outlet to charge an EV?",
    "answer": "Yes, but charging with a standard 120V outlet (Level 1) is very slow. A 240V Level 2 charger is recommended for daily use."
  },
  {
    "question": "What is vehicle-to-grid (V2G) technology?",
    "answer": "V2G allows EVs to send power back to the grid, helping balance energy demand and potentially lowering electricity costs."
  },
  {
    "question": "What brands make electric cars?",
    "answer": "Many brands make EVs, including Tesla, Ford, GM, Volkswagen, Hyundai, BYD, Rivian, and Lucid, among others."
  },
  {
    "question": "Will gas stations start offering EV charging?",
    "answer": "Yes, many gas stations are adding fast chargers as demand for EVs grows and fossil fuel use declines."
  }
    ])
    
    # Upload existing file and optionally append additional data
    converter.upload_and_append(existing_file_path, additional_json)
    
    # Print number of entries after upload
    print(f"Total entries after upload: {converter.get_entry_count()}")
    
    # View updated entries
    print("\nUpdated entries:")
    converter.view_entries(5)  # View first 5 entries

#Uncomment to run
if __name__ == "__main__":
    main()

Current entries before upload: 0
Successfully added 59 additional entries!
Total entries after upload: 319

Updated entries:
Entry 1: Question: How much does it cost to charge an electric vehicle at home?, Answer: It can cost between $40 to $53 per month if you pay the national average of 16 cents per kWh.
Entry 2: Question: What factors affect the cost of charging an electric vehicle at home?, Answer: The cost depends on how much you drive and the price you pay for residential electricity.
Entry 3: Question: What is the average cost of residential electricity in the U.S.?, Answer: The national average is 16 cents per kilowatt-hour (kWh).
Entry 4: Question: How much does charging an electric vehicle cost if electricity is priced at 25 cents per kWh?, Answer: The monthly cost would be between $63 to $83.
Entry 5: Question: How many miles does an electric vehicle typically drive per kWh?, Answer: An electric vehicle typically drives 3-4 miles per kWh.
